In [ ]:
import sys

sys.path.append("../")

from torch.utils.data import DataLoader
from copy import copy

from src.data import PeakWeatherTorchDataset, test_model, plot_predictions
from src.model import train_model, MLPModel

### Scegliere la finestra di osservazione e l'orizzonte di previsione
Vogliamo prevedere le prossime 24 ore, come con il modello ARIMA.

I modelli di deep learning spesso richiedono una finestra di osservazione più lunga per catturare le dinamiche temporali.
Scegliamo, per esempio, una finestra di 48 ore (2 giorni).

In [ ]:
horizon = 24  # Previsione a 24 ore per confrontare con ARIMA
window = 48

### Dividere il dataset in training, validation e test set
Creiamo tre dataset distinti per l'addestramento, la validazione e il test.

Il dataset di validazione viene utilizzato durante l'addestramento per monitorare le prestazioni del modello su dati non visti e stabilire quando fermare l'addestramento.

#### Domanda
Noti qualche differenza con la suddivisione del dataset rispetto a quella utilizzata per ARIMA?

In [ ]:
train_dataset = PeakWeatherTorchDataset(window=window, horizon=horizon)
val_dataset = copy(train_dataset)
val_dataset.mode = "val"
test_dataset = copy(train_dataset)
test_dataset.mode = "test"

# Creiamo i DataLoader per ciascun dataset
batch_size = 8192
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Scegliamo gli iperparametri della rete neurale
- hidden_size: numero di unità in ogni layer, larghezza della rete
- num_layers: numero di layer, profondità della rete
- lr: learning rate, velocità di apprendimento
- epochs: numero di volte in cui l'algoritmo vede l'intero dataset di training

In [ ]:
hidden_size = 16
num_layers = 2
lr = 0.001
epochs = 5

### Creiamo la rete neurale e addestriamola
Utilizziamo un modello MLP (Multi-Layer Perceptron) per la previsione della temperatura.
Questo modello prende in input una finestra di osservazioni e produce un'uscita per l'orizzonte di previsione specificato.

Gli MLP sono il tipo più classico di rete neurale.
Possono approssimare funzioni complesse tramite una sequenza di trasformazioni lineari e non lineari.

In [ ]:
model = MLPModel(window=window, horizon=horizon, hidden_size="???", num_layers="???")
model = train_model(
    model=model,
    lr="???",
    epochs="???",
    train_loader=train_dataloader,
    val_loader=val_dataloader,
)

### Testiamo il modello sui dati di test
Calcoliamo l'errore medio assoluto (MAE) del modello sui dati di test.

#### Domande
- Come si confronta questo valore con quello ottenuto con il modello ARIMA?
- I due valori sono direttamente confrontabili? Perché sì o perché no? (Suggerimento: quali serie temporali stiamo predicendo?)

In [ ]:
mae = test_model(model="???", test_loader=test_dataloader)

### Visualizziamo alcune previsioni del modello
Tracciamo alcune previsioni del modello sui dati di test per valutarne le prestazioni visivamente.

Puoi scambiare il numero di campioni visualizzati modificando il parametro `num_samples`.

In [ ]:
plot_predictions(model="???", test_dataset=test_dataset, num_samples=5)

#### Esercizio
- Prova a variare gli iperparametri del modello (window, hidden_size, num_layers, lr, epochs) e osserva come cambiano le prestazioni del modello.
- Cerca di migliorare il MAE ottenuto e di ottenere previsioni più accurate.
- Documenta le tue modifiche e i risultati ottenuti.

**Consiglio**: Modifica un iperparametro alla volta per capire meglio il suo impatto sulle prestazioni del modello.
Mantieni i valori sotto queste soglie per evitare lunghi tempi di addestramento:
- hidden_size ≤ 64
- num_layers ≤ 4
- epochs ≤ 20
- window ≤ 168